In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from typing import Dict, List, Tuple
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain


# --- 1. Load environment variables and define constants ---
load_dotenv()
MODEL = "openai/gpt-oss-120b"
db_name = "vector_db"

# --- 2. Set up API keys ---
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key:")

# --- 3. Document Loading and Processing ---
folders = glob.glob("knowledge-base/*")
text_loader_kwargs = {'encoding': 'utf-8'}
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([doc for doc in folder_docs])


# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['source'] for doc in documents)}")

# --- 4. Create Embeddings and Vector Store ---
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Clean up previous vector store if it exists
if os.path.exists(db_name):
    try:
        import shutil
        print(f"Attempting to delete previous vector store at '{db_name}'.")
        shutil.rmtree(db_name)
        print("Previous vector store deleted successfully.")
    except PermissionError as e:
        print(f"Warning: Could not delete the '{db_name}' directory due to a file lock. The process may be in use by another application or a previous run.")
        print("Please manually delete the folder or restart your Python kernel to resolve this issue.")

# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Created a chunk of size 1088, which is longer than the specified 1000


Total number of chunks: 123
Document types found: {'knowledge-base\\employees\\Alex Thomson.md', 'knowledge-base\\company\\about.md', 'knowledge-base\\employees\\Emily Tran.md', 'knowledge-base\\contracts\\Contract with EverGuard Insurance for Rellm.md', 'knowledge-base\\contracts\\Contract with Greenstone Insurance for Homellm.md', 'knowledge-base\\employees\\Jordan Blake.md', 'knowledge-base\\contracts\\Contract with Apex Reinsurance for Rellm.md', 'knowledge-base\\employees\\Avery Lancaster.md', 'knowledge-base\\employees\\Jordan K. Bishop.md', 'knowledge-base\\products\\Homellm.md', 'knowledge-base\\contracts\\Contract with GreenField Holdings for Markellm.md', 'knowledge-base\\contracts\\Contract with Stellar Insurance Co. for Rellm.md', 'knowledge-base\\employees\\Samuel Trenton.md', 'knowledge-base\\employees\\Alex Harper.md', 'knowledge-base\\contracts\\Contract with GreenValley Insurance for Homellm.md', 'knowledge-base\\contracts\\Contract with TechDrive Insurance for Carllm.

In [ ]:
# --- 5. Define LLM and Memory Store ---
llm = ChatGroq(model=MODEL, temperature=0.4, api_key=os.environ.get("GROQ_API_KEY"))
store: Dict[str, BaseChatMessageHistory] = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """
    Function to retrieve or create a chat history for a given session ID.
    This is required by RunnableWithMessageHistory.
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# --- 6. Create the RAG Chain with Memory ---
# This prompt is used to guide the retrieval. It takes the chat history into account.
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder("chat_history"),
    ("user", "{input}"),
    ("system",
     "Given the user's previous questions and the current question, "
     "rephrase the current question to be a standalone search query. Do NOT add any extra text."),
])

# Create a history-aware retriever
history_aware_retriever = create_history_aware_retriever(llm, vectorstore.as_retriever(), contextualize_q_prompt)

# This prompt is for the final answer generation, using retrieved documents.
qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. Use the following context to answer the user's question, "
     "and also use the chat history to maintain context. You sloud be able to solve logical queries also. \n\nContext: {context}"),
    MessagesPlaceholder("chat_history"),
    ("user", "{input}"),
])

# Create the document stuffing chain
stuff_documents_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create the final retrieval chain
rag_chain = create_retrieval_chain(history_aware_retriever, stuff_documents_chain)

# Add chat history to the entire chain
chain_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# --- 7. Gradio Chat Interface ---
def chat_response(message: str, history: List[Tuple[str, str]]) -> str:
    """
    Function to handle the chat interface logic.
    """
    # Create a unique session ID for the Gradio session.
    # The Gradio history is a list of lists, which is unhashable.
    # We must convert it to a tuple of tuples to be used as a session ID.
    session_id = str(hash(tuple(tuple(item) for item in history))) if history else "gradio_session"
    
    # Invoke the chain with the user's input and session config.
    response = chain_with_history.invoke(
        {"input": message},
        config={"configurable": {"session_id": session_id}}
    )
    
    return response['answer']

# Define the Gradio interface
iface = gr.ChatInterface(
    fn=chat_response,
    
    title="InsureLM ",
    description="Ask questions about the knowledge base.",
    examples=[
        "Details of Apex Reinsurance.",
        "Details of all the products made by Company",
        "Annual preformance history of Alex chen",
        "Tell me about the company"
    ]
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch()

c:\Users\Dell\anaconda3\Lib\site-packages\gradio\chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
